# Classifying Bangla Fake News with HuggingFace Transformers and Fastai


- toc: true
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In this post we cover fine tuning a [multilingual BERT](https://huggingface.co/bert-base-multilingual-cased) model from Huggingface Transformers library on [**BanFakeNews**](https://aclanthology.org/2020.lrec-1.349/) dataset released in LREC 2020. While English Fake News Classification and fact checking tasks have many resources and competitions available such as [fake news challenge](http://www.fakenewschallenge.org/) and [hateful meme detection](https://ai.facebook.com/blog/hateful-memes-challenge-and-data-set/), similar efforts in Bangla has been almost non existent. BanFakeNews dataset contains 50k annotated articles from different news sources in Bangladesh, out of them around 1200 articles have been annotated as fake. As transformer architectures uses self attention to learn contextual embeddings they have been very popular in NLP research community for a while and many tools have been built around them.

This post is reusing materials taught in Weights and Bias's study group of Fast ai with Huggingface([link](https://wandb.ai/wandb_fc/events/reports/W-B-Study-Group-Lectures-fast-ai-w-Hugging-Face--Vmlldzo4NDUzNDU?galleryTag=events)) where several recent(2021) libraries([blurr](https://github.com/ohmeow/blurr),[Fasthugs](https://github.com/morganmcg1/fasthugs) & [Adaptnlp](https://novetta.github.io/adaptnlp/)) that integrates components from popular deep learning frameworks Huggingface transformers and Fastai v2 are shown. 

My experience with using Transformers is fairly low, recently I participated in Huggingface's Flax/Jax week and there our team pretrained [Bengali GPT2](https://huggingface.co/flax-community/gpt2-bengali) and [T5](https://huggingface.co/flax-community/Bengali-t5/tree/main), so I was looking into Huggingface course and documentation to learn more about how to finetune transformers. Previously my thesis journal paper for undergraduate ["Deep Learning based Question Answering System in Bengali"](https://www.tandfonline.com/doi/full/10.1080/24751839.2020.1833136) worked on Bangla Question Answering with transformers, but I had stuck more to given scripts back then and focused more on synthetic dataset construction via translation and handling data corruption issues. So this post will focus more on the high level API of ```Blurr``` and the components of huggingface and fastai that are relevant for getting started quickly. 

In [3]:
!pip install -Uqq transformers datasets tqdm
!pip install -Uqq ohmeow-blurr
!pip install -Uqq wandb

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
import wandb

from transformers import *
from fastai.text.all import *
from fastai.callback.wandb import *


from blurr.data.all import *
from blurr.modeling.all import *

We will use weights and biases for tracking experiments and runs. Project page : https://wandb.ai/tasmiah-tahsin/fake-news-blurr

In [8]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Load dataset

This dataset has been downloaded from [Kaggle](https://www.kaggle.com/cryptexcode/banfakenews). Note that there are four files in the given dataset, but we use the two files (Authentic-48k and fake-1k) here because the rest of the files contain labelled fake and authentic news. Labelling in the sense of what type of fake news it is, clickbait, satire, misleading or false context, for the current priliminary phase we stick to binary classification of knowing if a news is fake or authentic. We also concatanate headline and content of the news article during preprocessing and combine the fake and authentic news dataset before sending them to Blurr dataloaders.



sample image goes here

In [10]:
fake = pd.read_csv("/content/drive/MyDrive/fake news/Fake-1K.csv")
authentic = pd.read_csv("/content/drive/MyDrive/fake news/Authentic-48K.csv",engine='python',error_bad_lines=False,warn_bad_lines=True)
df = pd.concat([authentic[['headline','content','label']],fake[['headline','content','label']]])
df.reset_index(drop=True,inplace=True)

In [11]:
print(authentic.shape, fake.shape)

(48678, 7) (1299, 7)


In [12]:
df['text'] = df['headline'] + df['content']

In [13]:
df = df.drop(['headline','content'],axis=1)

In [14]:
df.head(1)

,label,text
0,1,"হট্টগোল করায় বাকৃবিতে দুইজন বরখাস্ত, ৬ জনকে শোকজগত ১৭ সেপ্টেম্বর বাংলাদেশ কৃষি বিশ্ববিদ্যালয়ে (বাকৃবি) উপাচার্যের কার্যালয়ে হট্টগোলের ঘটনায় দুইজনকে সাময়িক বরখাস্ত ও ছয় জনকে শোকজ করেছে বিশ্ববিদ্যালয় প্রশাসন। বুধবার বিশ্ববিদ্যালয় বাকৃবি রেজিস্ট্রার সাইফুল ইসলাম স্বাক্ষরিত এক নোটিশে আগামী ৭ দিনের মধ্যে উপযুক্ত উত্তর দেয়ার নির্দেশ দেয়া হয়েছে। এদিকে এ ঘটনায় আন্দোলনের সঙ্গে একাত্বতা প্রকাশ না করায় হামলার শিকার হয়ে কারিগরি কর্মচারী পরিষদের সভাপতি ও সাধারণ সম্পাদক হাসপাতালে ভর্তি হয়েছেন। সাময়িক বরখাস্তরা হলেন- শিক্ষা বিষয়ক শাখার কর্মচারী ও ৩য় শ্রেণির সাধারণ সম্পাদক মো. মোশারফ হোসেন ও কর্মকর্তা পরি..."


In [15]:
df.label=df.label.map({1:"Authentic",0:"Fake"})

In [16]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.3)

We will use ```Blurr``` high level API for sequence classification with the pandas dataframe where ```BlearnerForSequenceClassification.from_dataframe()``` method takes in a dataframe, splits them from the column of default ```is_valid``` using Fastai's [```ColSplitter```](https://docs.fast.ai/data.transforms.html#ColSplitter) into train and test splits, constructs the datablock and dataloaders and uses them for training. So we add a ```'is_valid'``` column in the dataframe. There are other ways of splitting the data available in Fastai like ```RandomSubsetSplitter``` where we can randomize the data inside a dataframe. Since we used scikit-learns train test split to shuffle the dataframe for now we can go with Column based splitting. 

In [22]:
train['is_valid'] = False
valid['is_valid'] = True

SettingWithCopyError: ignored

In [23]:
final_df = pd.concat([train,valid],axis=0)

In [24]:
final_df.head()

,label,text,is_valid
2867,Authentic,"পুনম পান্ডের কাণ্ড!ফের সংবাদ শিরোনামে পুনম পান্ডে। এশিয়া কাপে ভারত-পাকিস্তানের ম্যাচ উপলক্ষে পুনম পান্ডে যা করলেন, তা অবাক করা কাণ্ড বটে। বুধবার ছিল ভারত-পাকিস্তানের ক্রিকেট ম্যাচ। আর সেই ম্যাচের আগেই সোশ্যাল সাইটে নিজের ছবি শেয়ার করলেন পুনম। যেখানে তাঁর শরীরের বেশ কিছুটা অংশে এক কুচি সুতোর দেখা মেলেনি। অর্থাত, শরীরের নিন্মাংশ প্রদর্শন করে সোশ্যাল সাইটে উত্তাপ ছড়াতে শুরু করেন পুনম। শুধু তাই নয়, এবারই মজা হবে বলেও নিজের ছবির নিচে ক্যাপশন দিতেও দেখা যায় পুনমকে। ‘ইন্টারনেট সেনসেশন’ পুনমের সেই ছবি প্রকাশ্যে আসতেই তা ভাইরাল হয়ে যায় সঙ্গে সঙ্গে। তবে এই প্রথমবার নয়, এর আগেও বহুবার নিজের সোশ্যাল ...",False
49122,Fake,যে ১০টি অতিমানবীয় দক্ষতার কারণে ঢাকাবাসীর নোবেল পুরস্কার পাওয়া উচিত - Bengal Beatsঢাকাবাসীর জীবনের খাতায় একের পর এক এত সম্মান এসে জুটছে যে আর কিছুদিন পর শুধুমাত্র নিজেদেরকে এমন সম্মানী শহরের একজন গর্বিত নাগরিক হিসেবে দেখতে পৃথিবীর আর দশটা উন্নত শহর থেকে সবাই এখানে চলে আসবে কিনা সেটাই এখন ভাববার বিষয়। তার উপর নাসার উচ্চপদস্থ গবেষকরা জানিয়েছেন যে এখন থেকে ঢাকার মানুষদের নিশ্চিন্তে মহাবিশ্বের যেকোনো স্থানে পাঠানো সম্ভব। কারণ এরা নিঃসন্দেহে যেকোনো দুর্গম গ্রহে টিকে থাকতে পারবে। তাই অতি দ্রুত এদেরকে সম্মানিত করতে আমাদের সকলের উচিত ঢাকাবাসীকে নোবেল পুরস্কার দেয়া। কিন্তু কেন দেয়া উচিত? চলুন তাহলে...,False
14531,Authentic,"মুন্সীগঞ্জে ‘বন্দুকযুদ্ধে’ মাদক কারবারি নিহত ২৬ সেপ্টেম্বর, ২০১৮ ১০:৩২মুন্সীগঞ্জে র‌্যাবের সঙ্গে ‘বন্দুকযুদ্ধে’ আব্দুল মালেক নামের এক মাদক কারবারি নিহত হয়েছে। র‌্যাবের দাবি, এ সময় তাদের দুইজন সদস্য এএসআই মোর্শেদ ও নায়েক লোকমান আহত হয়েছে। র‌্যাব-১১ এর কোম্পানী কমান্ডার মোহিতুল ইসলাম জানান, মঙ্গলবার (২৫ সেপ্টেম্বর) দিবাগত রাত সাড়ে ৩টার দিকে মিরকাদিম পৌরসভার নগর কসবা এলাকায় তাদের টহল টিমের সাথে একদল মাদক কারবারিদের ১৫ মিনিট স্থায়ী একটি বন্দুক যুদ্ধ চলে। এক পর্যায়ে মাদক কারবারিরা পিছু হটে গেলে ঘটনা স্থলে মাদক কারবারি আব্দুল মালেকের মৃত্য দেহ পরে থাকতে দেখে তারা। এছাড়া ঘটনা স্থল থেকে, একটি বিদে...",False
45451,Authentic,"ব্রিটিশ এমপিদের কার্যালয়ে ব্যবহৃত কনডোম, বিরক্ত পরিচ্ছন্নতা কর্মীরাব্রিটিশ এমপিদের বিরুদ্ধে ভয়াবহ অভিযোগ এনেছেন তাদের কার্যালয় পরিস্কারের দায়িত্বে থাকা পরিচ্ছন্নতা কর্মীরা। তারা অভিযোগ করে জানিয়েছেন, ব্রিটিশ এমপিদের কার্যালয়ে প্রতিনিয়ত তারা ব্যবহৃত কনডোম এবং বমি দেখতে পান। আর এটা পরিস্কার করতে করতে তারা বিরক্ত। দ্য সানডে টাইমসের এক প্রতিবেদনে জানানো হয়েছে, পার্লামেন্টে ব্রিটিশ এমপিদের অফিস আর পরিস্কার করতে চাইছেন না পরিচ্ছন্নতা কর্মীরা৷ কারণ রোজ অফিসে ঢুকলে বমি আর ব্যবহৃত কন্ডোম খুঁজে পান তারা৷ সূত্রের বরাত দিয়ে সানডে টাইমসের প্রতিবেদনে বলা হয়েছে, এই ধরনের আচরণ কলেজ ছাত্রছাত্রীদের মানায়৷ ...",False
48513,Authentic,"'৩৫ বছর বয়সেও আর্জেন্টিনাকে বিশ্বকাপ জেতার ক্ষমতা রাখেন মেসি'দায়িত্ব ছেড়ে চলে গেছেন অনেকদিন হলো। তবে ছাত্রের উপর এখনও আস্থা রাখছেন শিক্ষক। সাবেক আর্জেন্টাইন কোচ জর্জ সাম্পাওলি মনে করছেন, মেসির এখনও বিশ্বকাপ জেতার ক্ষমতা আছে। বয়স কোনো ফ্যাক্টর নয়। আত্মবিশ্বাস থাকলে দুর্গমও সুগম হয়। শুধু বিশ্বাস করতে হবে, ‘তুমি পারবে’। আর বিশ্ব ফুটবলের উজ্জ্বলতম নক্ষত্রের সেই আত্মবিশ্বাস আছে বলেই মনে করেন সাম্পাওলি। তিনি বিশ্বাস করেন ৩৫ বছর বয়সেও লিওনেল মেসি আর্জেন্টিনাকে বিশ্বকাপ জেতাতে পারবেন। প্রসঙ্গত, রাশিয়া বিশ্বকাপে স্বপ্নভঙ্গ হওয়ার পর দীর্ঘ দিন দেশের সাথে যোগাযোগ বিছিন্ন মেসির। এমনকি রাশিয়া থেকে বাড়ি...",False


# Model

## Multilingual BERT

Since the original paper also used [multilingual cased bert](https://github.com/google-research/bert/blob/master/multilingual.md) released by Google this post can be considered as an attempt to reproduce the work of BanFakeNews. They trained mbert for 50 epochs with a learning rate of 0.00002 and optimizer Adam. The batch size was 32. The overall F1 score after training for 50 epochs on this dataset was .99 and f1 for fake class was 0.68. Multilingual bert has been pretrained on 104 languages including bengali with wordpiece tokenization. As bengali is already included it makes it a valid choice for current bangla text classification task. Information for this model are : 104 languages, 12-layer, 768-hidden, 12-heads, 110M parameters.  As the size of the language corpora varied greatly for low resource languages exponential weighted smoothing was performed for weighting data during the pretraining stage, which results in undersampling high resource languages like english and oversampling low resource languages like Bengali. Mbert does not use any marker for input language to enable zero shot training.

In [25]:
wandb_init_kwargs = {
    'reinit': True, 
    'project': "fake-news-blurr", 
    'entity': "tasmiah-tahsin",
    'notes': 'Finetuning banfakenews with multilingual bert via Blurr', 
    'tags': ['mbert', 'fake-news-classification', 'blurr']
  }

In [26]:
wandb.init(**wandb_init_kwargs)

wandb: Currently logged in as: tasmiah-tahsin (use `wandb login --relogin` to force relogin)


Since I'm fairly new in ```blurr``` I'm using the high level API, but the key ideas are following. ```blurr``` is integrating two frameworks. Here ```Fastai``` is providing the ```datablock```, ```learner```, ```learning rate finder``` functionalities with Leslie Smith's 1cycle policy components, while huggingface transformers is providing the ready to use transformer model configuration and architectures made publicly available from huggingface hub. 

Fastai's datablock helps to quickly transform a raw dataset to a mini-batch that can be sent to a model for training with its various transforms and splitting functions along with visualization capacities. Under the hood ```blurr``` is providing wrappers for the huggingface transformers and for finetuning its also grouping the parameters of the transformers model along with enabling discriminative fine tuning like  [ULMFiT](https://paperswithcode.com/method/ulmfit). Discriminative finetuning refers to using variable learning rates for different layers of a neural network and they can also change during training(increasing or decreasing learning rate will change how much the parameters are updated during each step).

Datablock transforms can operate on both item and batch level. Fastai also provides the capacity add callbacks in the training loop. Callbacks are a generic programming concept which refers to calling a function inside another function(the base function calls the callback function, hence the naming) and are extensively used in DL training loops to perform specific tasks during the forward pass, loss calculation, backward pass and optimizer step taking for issues like changing learing rates and more. 

The low level API for ```blurr```  works with datablocks and dataloaders  creating the mini batches which are combined with the model architecture, optimizer and loss function inside a ```learner```. Mid level API contains ```BLearner``` and the highest level API contains task specific learners like the current one I'm using which is ```BlearnerForSequenceClassification```.  




In [ ]:
from blurr.modeling.core import BlearnerForSequenceClassification
pretrained_model_name = "bert-base-multilingual-cased"
learn = BlearnerForSequenceClassification.from_dataframe(final_df, pretrained_model_name, dl_kwargs={ 'bs': 4})

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
learn.lr_find(suggest_funcs=[minimum, steep, valley, slide])


In [ ]:
learn.freeze()
learn.fit_one_cycle(1, lr_max=1e-2,cbs=[WandbCallback(log_preds=False, log_model=False)])

Could not gather input dimensions


epoch,train_loss,valid_loss,f1_score,accuracy,time


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, lr_max=1e-2,cbs=[WandbCallback(log_preds=False, log_model=False)])

In [ ]:
learn.show_results(learner=learn, max_n=4,trunc_at=200)

,text,category,target
0,"আফগানদের ছিটকে বাংলাদেশের রুদ্ধশ্বাস উত্তেজনার ম্যাচে ৩ রানে জিতেছে বাংলাদেশ । ২৪৯ রান আফগানিস্তান থেমেছে ২৪৬ রানে । সংক্ষিপ্ত স্কোর : বাংলাদেশ : ৫০ ওভারে ২৪৯ / ৭ ( লিটন ৪১, শান্ত ৬, মিঠুন ১, মুশফিক ৩",Authentic,Authentic
1,"এলেবেলে : খাল কেটে কুমীর আনবেন না, সিদ্দিককে ভোট দেবেন না [ আহমেদের'এলেবেলে'বাংলা রম্য সাহিত্যে এক অনন্য সংযোজন । এর শিরোনামের রচনাগুলো তিনি তার ছোট ভাই আহসান হাবীবের প্রকাশিত মাসিক উন্মাদের জন্য লিখা",Fake,Fake
2,"বিএনপি শুদু লাঞ্ছনা বঞ্ছনা গঞ্জনা : শমশের | দৈনিক মতিকণ্ঠনিজস্ব মতিবেদকরাজনীতী হতে অবসর ঘোষনা বৃহত্তর ইসলামীর বিএনপি শাখার খারিজি, বৃহত্তর ইসলামীর বিএনপি শাখার মহিলা আমীর ও শক্তির মালিক আপোষহীন দেশনেত",Fake,Fake
3,"বোকো হারাম - গন্তব্য কতদূর? বোকো হারাম উগ্রপন্থী দল । পশ্চিম প্রথমে গঠিত জামাত আহলে সুন্নাহ লিদ জিহাদ, এটিই কিছুদিন পর বোকো হারাম নামে পরিচিতি । এদের মূল বক্তব্য হলো, পশ্চিমা শিক্ষা - সংস্কৃতি নিষিদ্ধ",Authentic,Authentic


In [ ]:
wandb.finish()

epoch,5
train_loss,0.08289
raw_loss,0.07101
wd_0,0.01
sqr_mom_0,0.99
lr_0,0.0
mom_0,0.95
eps_0,1e-05
wd_1,0.01
sqr_mom_1,0.99
lr_1,0.0


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train_loss,▇██▇▆▅▅▅▅▅▃▅▄▄▂▄▄▃▃▃▂▂▃▃▁▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁
raw_loss,▇▃▃▇▃▃▆▂▃▅▁▆▆▁▁▇▁▁▆▄▅▁▃▄▁▆▂▄▂▃▂▁▄▁▁▁▁█▁▂
wd_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁
mom_0,██▇▆▅▄▃▂▁▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██████
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wd_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sqr_mom_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_1,▁▂▂▃▄▅▆▇███████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁


# Model Sharing

In [ ]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,676 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 160837 files and directories c

In [ ]:
!transformers-cli login

2021-08-05 07:17:22.031785: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: Tahsin-Mayeesha
Password: 
Login successful
Your token: oSBeuCeqFwuZgHGyqIBjgHEmiAchBfqCTsadczfjKMwdGdOskZjIvLzkFxbQafNAbNHwgsNibftTGdtUfMvWLkBjGwsMGfpdekOyyFbWFVOOJUkktSsdoacPJeeuZkhi 

Your token has been saved to /root/.huggingface/token


In [ ]:
!git config --global user.email "tasmiah.tahsin@northsouth.edu"

In [ ]:
!git config --global user.name "Tahsin-Mayeesha"

In [ ]:
blurr_tfm = get_blurr_tfm(learn.dls.before_batch)

In [ ]:
blurr_tfm.hf_model.push_to_hub("bangla-fake-news-mbert",use_temp_dir=True)

'https://huggingface.co/Tahsin-Mayeesha/bangla-fake-news-mbert/commit/2485b1ab7750b7746e6a052bf72f6c8871e8ecd4'

In [ ]:
blurr_tfm.hf_tokenizer.push_to_hub("bangla-fake-news-mbert",use_temp_dir=True)

PanicException: ignored

# Explaining predictions

In [ ]:
sample news = 

In [ ]:
!pip install transformers-interpret


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "Tahsin-Mayeesha/bangla-fake-news-mbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# With both the model and tokenizer initialized we are now able to get explanations on an example text.

from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
word_attributions = cls_explainer("I love you, I like you")


In [ ]:
cls_explainer.visualize